In [1]:
%load_ext tensorboard
import warnings
warnings.filterwarnings("ignore")
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense,RNN,Flatten, Softmax
from tensorflow.keras import layers
from tensorflow.keras.models import model_from_json

from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from tensorflow.keras import initializers, regularizers, constraints
import numpy as np
import datetime

import pickle
import shutil
from IPython.display import Image
import io
from nltk.translate.bleu_score import sentence_bleu

In [2]:
print(tf. __version__)

2.11.0


In [3]:
from platform import python_version

print(python_version())

3.8.10


In [4]:

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/AI/Datasets/Deep Text Corrector')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
os.getcwd()

'/content/drive/MyDrive/AI/Datasets/Deep Text Corrector'

In [6]:
f = open('./data/perturbated_text_embed_matrix.pkl','rb')
perturbated_text_embed_matrix = pickle.load(f)
f.close()

f = open('./data/text_embed_matrix.pkl','rb')
text_embed_matrix = pickle.load(f)
f.close()

f = open('./data/perturbated_text_train.pkl','rb')
perturbated_text_train = pickle.load(f)
f.close()

f = open('./data/text_inp_train.pkl','rb')
text_inp_train = pickle.load(f)
f.close()

f = open('./data/text_out_train.pkl','rb')
text_out_train = pickle.load(f)
f.close()

f = open('./data/perturbated_text_tokernizer_index.pkl','rb')
perturbated_text_tokernizer_index = pickle.load(f)
f.close()

f = open('./data/text_inp_tokernizer_word_index.pkl','rb')
text_inp_tokernizer_word_index = pickle.load(f)
f.close()

f = open('./data/perturbated_text_tokernizer.pkl','rb')
perturbated_text_tokernizer = pickle.load(f)
f.close()


f = open('./data/text_inp_tokernizer.pkl','rb')
text_inp_tokernizer = pickle.load(f)
f.close()

train_data = pd.read_csv('./data/train_data.csv', usecols=['enc_inp', 'dec_inp', 'dec_out'])
test_data = pd.read_csv('./data/test_data.csv', usecols=['enc_inp', 'dec_inp', 'dec_out'])
validation_data = pd.read_csv('./data/validation_data.csv', usecols=['enc_inp', 'dec_inp', 'dec_out'])


In [7]:
train_data

,enc_inp,dec_inp,dec_out
0,fred fred actually been on an extended vacat...,sos fred fred he is actually he is been on an ...,fred fred he is actually he is been on an exte...
1,no funniest show that mickey and i ever did o...,sos no the funniest show that mickey and i eve...,no the funniest show that mickey and i ever di...
2,do not send flowers joe i am not dead yet,sos do not send flowers joe i am not dead yet,do not send flowers joe i am not dead yet eos
3,idea was out there in universe now what,sos the idea was out there in the universe now...,the idea was out there in the universe now wha...
4,i do not recommend it,sos i do not recommend it,i do not recommend it eos
...,...,...,...
163663,you know sometimes people can surprise you som...,sos you know sometimes people can surprise you...,you know sometimes people can surprise you som...
163664,yeah chili hi you are fifty feet in the air,sos yeah chili hi you are fifty feet in the air,yeah chili hi you are fifty feet in the air eos
163665,incredible the cloth the buttons it looks to b...,sos incredible the cloth the buttons it looks ...,incredible the cloth the buttons it looks to b...
163666,it is not me moron,sos it is not me moron,it is not me moron eos


In [8]:
print(len(perturbated_text_train))
print(len(text_inp_train))
print(len(text_out_train))

print(perturbated_text_train.shape)
print(text_inp_train.shape)
print(text_out_train.shape)

print(len(text_embed_matrix))
print(len(perturbated_text_embed_matrix))

163668
163668
163668
(163668, 20)
(163668, 20)
(163668, 20)
34604
34602


In [9]:
class Encoder(tf.keras.Model):
    '''
    Encoder- Takes a input sequence and returns output sequence
    '''
    def __init__(self,input_vocab_size,embedd_size,lstm_size,inp_len):
        super().__init__()

        self.input_vocab_size = input_vocab_size
        self.embedd_size = embedd_size
        self.lstm_size = lstm_size
        self.inp_len = inp_len

        self.embedd = Embedding(input_dim = self.input_vocab_size, output_dim = self.embedd_size, input_length=self.inp_len,
                                weights = [perturbated_text_embed_matrix], mask_zero=True)
        self.encoder_lstm = LSTM(units = self.lstm_size, return_sequences=True, return_state=True, 
                               name="Encoder", kernel_regularizer= regularizers.l2(1e-5))

    def call(self,inp_seq, training=True):
        embedds = self.embedd(inp_seq)
        encoder_output, encoder_hidden_state, encod_cell_state = self.encoder_lstm(embedds)
        return encoder_output, encoder_hidden_state, encod_cell_state

    def initialize_states(self,batch_size):
        h_state = np.zeros((batch_size, self.lstm_units))
        c_state = np.zeros((batch_size, self.lstm_units))
        return h_state, c_state


In [10]:
#https://github.com/UdiBhaskar/TfKeras-Custom-Layers/blob/master/Seq2Seq/clayers.py
class MonotonicBahadanauAttention(tf.keras.layers.Layer):
    def __init__(self, units,
                 return_aweights=False,
                 scaling_fact=None,
                 noise_std=0,
                 weights_inits='he_normal',
                 bias_inits='zeros',
                 **kwargs):
        
        if 'name' not in kwargs:
            kwargs['name'] = ""
            
        super(MonotonicBahadanauAttention, self).__init__(**kwargs)
        self.units = units
        self.scaling_fact = scaling_fact
        self.noise_std = noise_std
        self.weights_inits = initializers.get(weights_inits)
        self.bias_inits = initializers.get(bias_inits)
        self.weights_regs = regularizers.l2(1e-2)
    
    def build(self, inp_shape):
        self._wa = layers.Dense(self.units, use_bias=False,\
            kernel_inits=self.weights_inits, bias_inits=self.bias_inits,\
                kernel_regs= self.weights_regs, name=self.name+"Wa")
        
        self._ua = layers.Dense(self.units,\
            kernel_inits=self.weights_inits, bias_inits=self.bias_inits,\
                kernel_regs= self.weights_regs, name=self.name+"Ua")
        
        self._va = layers.Dense(1, use_bias=False, kernel_inits=self.weights_inits,\
            kernel_regs= self.weights_regs,bias_inits=self.bias_inits, name=self.name+"Va")
        
        
    def call(self, dec_h_state, enc_out, previous_attention, training=True):

        enc_out, dec_h_state = tf.cast(enc_out, tf.float32), \
            tf.cast(dec_h_state, tf.float32)
        
        dec_h_with_time_axis = tf.expand_dims(dec_h_state, 1)

        weight_wa=self._wa
        weight_ua=self._ua
        weight_va=self._va
        
        #bahdanau attention score
        score = weight_va(tf.nn.tanh(weight_wa(dec_h_with_time_axis) + weight_ua(enc_out)))
        score = tf.squeeze(score, [2])
        
        if self.scaling_fact is not None:
            score = score/tf.sqrt(self.scaling_fact)

        if training:
            if self.noise_std > 0:
                random_noise = tf.random.normal(shape=tf.shape(input=score), mean=0,\
                    stddev=self.noise_std, dtype=score.dtype, seed=self.seed)
                score = score + random_noise

        probs = tf.sigmoid(score)

        #monotonic attention 'parallel' mode
        cumprod_1mp_probs = tf.exp(tf.cumsum(tf.math.log(tf.clip_by_value(1-probs,\
            1e-10, 1)), axis=1, exclusive=True))
        att_weights = probs*cumprod_1mp_probs*tf.cumsum(previous_attention/\
            tf.clip_by_value(cumprod_1mp_probs, 1e-10, 1.), axis=1)
        att_weights = tf.expand_dims(att_weights, 1)

        contxt_vec = tf.matmul(att_weights, enc_out)
        contxt_vec = tf.squeeze(contxt_vec, 1, name="contxt_vec")

        return contxt_vec, tf.squeeze(att_weights, 1, name='att_weights')


In [11]:
class OneStepDecoder(tf.keras.Model):

    def __init__(self,target_vocab_size, embedd_dim, inp_len, decoder_units, attention_units):
        super().__init__()
        
        self.target_vocab_size = target_vocab_size
        self.embedd_dim = embedd_dim
        self.inp_len = inp_len
        self.decoder_units = decoder_units
        self.attention_units = attention_units
        
        self.decoder_embedding_layer = Embedding(input_dim = self.target_vocab_size, output_dim = self.embedd_dim,
                                                 input_length = self.inp_len,
                                      weights = [text_embed_matrix] , name="onestepdecoder_embedding_layer", mask_zero=True)
        
        self.decoder_LSTM = LSTM(units = self.decoder_units, return_state=True, kernel_regularizer= regularizers.l2(1e-5))
        
        self.MonotonicBahadanauAttention=MonotonicBahadanauAttention(self.attention_units,
                 return_aweights=False,
                 scaling_factor=None,
                 noise_std=0,
                 weights_initializer='he_normal',
                 bias_initializer='zeros',)

        self.dense = Dense(units=self.target_vocab_size)

    def call(self,inp_to_dec, encoder_output, state_hidden, state_cell, att_weights):
        
        decoder_embedd = self.decoder_embedding_layer(inp_to_dec)
        context_vec, att_weights = self.MonotonicBahadanauAttention(state_hidden,encoder_output, att_weights)
        decoder_embedd = tf.concat([tf.expand_dims(context_vec,1), decoder_embedd], axis=-1)
        decoder_out, decoder_hidden_state, decoder_cell_state = self.decoder_LSTM(decoder_embedd, 
                                                                                initial_state=[state_hidden, state_cell])
        onestep_decoder_output = self.dense(decoder_out)

        return onestep_decoder_output, decoder_hidden_state, decoder_cell_state, att_weights, context_vec


In [12]:
class Decoder(tf.keras.Model):
    def __init__(self,output_vocab_size, embedd_dim, output_length, decoder_units , att_units):
        super().__init__()

        self.output_vocab_size = output_vocab_size
        self.embedd_dim = embedd_dim
        self.output_length = output_length
        self.decoder_units = decoder_units
        self.att_units = att_units

        self.onestep_decoder = OneStepDecoder(self.output_vocab_size, self.embedd_dim, self.output_length, self.decoder_units
                                              ,self.att_units)

        
    def call(self, inp_to_dec,enc_out,decoder_h,decoder_c, att_weights ):

        total_out = tf.TensorArray(tf.float32, size=tf.shape(inp_to_dec)[1], name='out_arrays')
        i = tf.shape(inp_to_dec)[1]

        for t_step in range(i):    

            onestep_decoder_output, decoder_h, decoder_c, att_weights, context_vector = self.onestep_decoder(
                                            inp_to_dec[:, t_step:t_step+1], enc_out, decoder_h, decoder_c, att_weights)

            total_out = total_out.write(t_step, onestep_decoder_output)
        
        total_out = tf.transpose(total_out.stack(), [1,0,2])

        return total_out


In [13]:
class bahadanau_attention_model(tf.keras.Model):
    def __init__(self,encoder_vocab_size, decoder_vocab_size, encoder_embedd_dim, decoder_embedd_dim, input_len, output_len, 
                 encoder_units, decoder_units, attention_units):
        super().__init__()

        self.encoder_vocab_size = encoder_vocab_size
        self.decoder_vocab_size = decoder_vocab_size
        self.encoder_embedd_dim = encoder_embedd_dim
        self.decoder_embedd_dim = decoder_embedd_dim
        self.input_len = input_len
        self.output_len = output_len
        self.encoder_units = encoder_units
        self.decoder_units = decoder_units
        self.attention_units = attention_units

        self.encoder = Encoder(self.encoder_vocab_size,self.encoder_embedd_dim,self.encoder_units,self.input_len)
        self.decoder = Decoder(self.decoder_vocab_size,self.decoder_embedd_dim,self.output_len,self.decoder_units,
                               self.attention_units)

    def call(self, data, training=True):
        encoder_inp, decoder_inp = data[0], data[1]
        
        encoder_out, encoder_h, encoder_c = self.encoder(encoder_inp)
    
        decoder_h = encoder_h  #initial decoder state is equal to final encoder hidden state
        decoder_c = encoder_c

        att_weights = np.zeros((512, 20), dtype='float32')
        att_weights[:, 0] = 1
        
        final_output = self.decoder(decoder_inp,encoder_out,decoder_h,decoder_c, att_weights)
        return final_output


In [14]:
class Dataset:
    def __init__(self, data, perturbed_tkn, corr_tkn, max_len):
        self.enc_inputs = data['enc_inp'].values
        self.dec_inputs = data['dec_inp'].values
        self.dec_outputs = data['dec_out'].values
        self.corr_tkn = text_inp_tokernizer
        self.perturbed_tkn = perturbated_text_tokernizer
        self.max_len = max_len

    def __getitem__(self, i):
        #self.enc_sequence = self.perturbed_tkn.texts_to_sequences([self.enc_inputs[i]]) # need to pass list of values
        #self.dec_input_sequence = self.corr_tkn.texts_to_sequences([self.dec_inputs[i]])
        #self.dec_output_sequence = self.corr_tkn.texts_to_sequences([self.dec_outputs[i]])

        #self.enc_sequence = pad_sequences(self.enc_sequence, maxlen=self.max_len, dtype='float32', padding='post')
        #self.dec_input_sequence = pad_sequences(self.dec_input_sequence, maxlen=self.max_len, dtype='float32', padding='post')
        #self.dec_output_sequence = pad_sequences(self.dec_output_sequence, maxlen=self.max_len, dtype='float32', padding='post')
          
        self.enc_sequence=[perturbated_text_train[i]]
        self.dec_input_sequence=[text_inp_train[i]]
        self.dec_output_sequence=[text_out_train[i]]
        
        return self.enc_sequence, self.dec_input_sequence, self.dec_output_sequence

    def __len__(self): # your model.fit_gen requires this function
        return len(self.enc_inputs)
    
class Dataloader(tf.keras.utils.Sequence):
    
    def __init__(self, data, batch_size=1):
        self.data = data
        self.batch_size = batch_size
        self.indices = np.arange(len(self.data.enc_inputs))


    def __getitem__(self, i):
        begin = i * self.batch_size
        end = (i + 1) * self.batch_size
        data_sequence= []
        for j in range(begin, end):
            data_sequence.append(self.data[j])

        batch = [np.squeeze(np.stack(seq, axis=1), axis=0) for seq in zip(*data_sequence)]
        
        return [batch[0],batch[1]],batch[2]

    def __len__(self):  # your model.fit_gen requires this function
        return len(self.indices) // self.batch_size

    def on_epoch_end(self):
        self.indices = np.random.permutation(self.indices)

In [15]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

train_data_padded = Dataset(train_data, perturbated_text_tokernizer_index, text_inp_tokernizer, 20)
validation_data_padded  = Dataset(validation_data, perturbated_text_tokernizer_index, text_inp_tokernizer, 20)
test_data_padded = Dataset(test_data, perturbated_text_tokernizer_index, text_inp_tokernizer, 20)

batch_train_data = Dataloader(train_data_padded, batch_size=512)
batch_validation_data = Dataloader(validation_data_padded, batch_size=512)
batch_test_data = Dataloader(test_data_padded, batch_size=512)

print(batch_train_data[0][0][0].shape, batch_train_data[0][0][1].shape, batch_train_data[0][1].shape)

(512, 20) (512, 20) (512, 20)


In [16]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

In [17]:
encoder_vocab_size = len(perturbated_text_embed_matrix)
decoder_vocab_size = len(text_embed_matrix) 
encoder_embedd_dim = 300
decoder_embedd_dim = 300
input_len = 20
output_len = 20
encoder_units = 256
decoder_units = 256
att_units = 256

In [38]:
optimizer = tf.keras.optimizers.Adam(clipnorm=1)
log_directory = os.getcwd() + '/bahadanau_attention_model/logs/fit/' + str(datetime.datetime.now())
checkpoint_model = ModelCheckpoint("bahadanau_attention_model_weights_"+ str(datetime.datetime.now())+".hdf5", 
                                   monitor='loss', save_best_only=True, save_weights_only=True, verbose=0, mode='min')
earlystopping_model = EarlyStopping(monitor='loss', patience=10, verbose=1)
tensorboard = TensorBoard(log_dir=log_directory, histogram_freq=1)
callbacks_model = [checkpoint_model, tensorboard, earlystopping_model]

In [39]:
model = bahadanau_attention_model(encoder_vocab_size, decoder_vocab_size, encoder_embedd_dim, decoder_embedd_dim, 
                              input_len, output_len, encoder_units, decoder_units ,att_units)

In [40]:
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [22]:
train_steps=train_data.shape[0]//512
valid_steps=validation_data.shape[0]//512
model.fit(batch_train_data, steps_per_epoch=train_steps, epochs=20, validation_data=batch_validation_data,
                     validation_steps=valid_steps, callbacks=[callbacks_model])

Epoch 1/20
319/319 [==============================] - 269s 773ms/step - loss: 4.3607 - accuracy: 0.0647 - val_loss: 2.4700 - val_accuracy: 0.0717
Epoch 2/20
319/319 [==============================] - 222s 697ms/step - loss: 2.2582 - accuracy: 0.1004 - val_loss: 2.0626 - val_accuracy: 0.1246
Epoch 3/20
319/319 [==============================] - 228s 716ms/step - loss: 1.8915 - accuracy: 0.1511 - val_loss: 1.6988 - val_accuracy: 0.1844
Epoch 4/20
319/319 [==============================] - 223s 698ms/step - loss: 1.4642 - accuracy: 0.2299 - val_loss: 1.2559 - val_accuracy: 0.2709
Epoch 5/20
319/319 [==============================] - 225s 704ms/step - loss: 1.0958 - accuracy: 0.3003 - val_loss: 0.9536 - val_accuracy: 0.3235
Epoch 6/20
319/319 [==============================] - 224s 702ms/step - loss: 0.8701 - accuracy: 0.3371 - val_loss: 0.7715 - val_accuracy: 0.3506
Epoch 7/20
319/319 [==============================] - 226s 707ms/step - loss: 0.7065 - accuracy: 0.3612 - val_loss: 0.6269 -

In [35]:
model(tf.ones(shape=(512,512,20)))
model.load_weights(os.getcwd() + "/bahadanau_attention_model_weights.hdf520230214-075454")

In [36]:
model.fit(batch_train_data, steps_per_epoch=train_steps, epochs=5, validation_data=batch_validation_data,
                      validation_steps=valid_steps, callbacks=[callbacks_model])

Epoch 1/5
319/319 [==============================] - 226s 705ms/step - loss: 0.1058 - accuracy: 0.4404 - val_loss: 0.1059 - val_accuracy: 0.4391
Epoch 2/5
319/319 [==============================] - 224s 701ms/step - loss: 0.0978 - accuracy: 0.4411 - val_loss: 0.1014 - val_accuracy: 0.4397
Epoch 3/5
319/319 [==============================] - 224s 702ms/step - loss: 0.0877 - accuracy: 0.4425 - val_loss: 0.0966 - val_accuracy: 0.4398
Epoch 4/5
319/319 [==============================] - 222s 697ms/step - loss: 0.0928 - accuracy: 0.4411 - val_loss: 0.0995 - val_accuracy: 0.4386
Epoch 5/5
319/319 [==============================] - 223s 700ms/step - loss: 0.0810 - accuracy: 0.4429 - val_loss: 0.0944 - val_accuracy: 0.4389


In [57]:
#%tensorboard --log_directory logs/fit

In [44]:
encoder_layer = model.layers[0]
decoder_layer = model.layers[1]
onestepdecoder = decoder_layer.layers[0]

def EncoderOutput(enc_input):
    encoder_input=list()
    for word in enc_input.split():
        if perturbated_text_tokernizer_index.get(word) != None:
            encoder_input.append(perturbated_text_tokernizer_index.get(word))
        else:
            encoder_input.append(0)
    
    encoder_input = tf.keras.preprocessing.sequence.pad_sequences([encoder_input], maxlen=20, padding='post')
    encoder_output, encoder_hidden_state, encoder_cell_state = encoder_layer(encoder_input,0)
    
    return encoder_output, encoder_hidden_state, encoder_cell_state


def PredictOutput(enc_inp, dec_inp):
    enc_out, enc_hidden, enc_cell=EncoderOutput(enc_inp)
    dec_hidden = enc_hidden
    dec_cell = enc_cell 
    translated_output=""
    dec_inp = tf.expand_dims([text_inp_tokernizer.word_index['sos']],0)

    att_weights = np.zeros((1,20), dtype='float32')
    att_weights[:,0] = 1    
    
    for word in range(20):

            predicted, dec_hidden, dec_cell, att_weights, context_vec = onestepdecoder(dec_inp, enc_out,
                                                                            dec_hidden, dec_cell, att_weights)        

            predicted_word_index = tf.argmax(predicted[0]).numpy()

            if text_inp_tokernizer.index_word[predicted_word_index] == 'eos':
                return translated_output 

            translated_output = translated_output + text_inp_tokernizer.index_word[predicted_word_index] + ' '
            dec_inp = tf.expand_dims([predicted_word_index], 0)

    return translated_output

def InferResults(data):
    output = []

    for enc_inp, dec_inp, dec_out in data.values:
        pred = PredictOutput(enc_inp, dec_inp)
        output.append(pred)

    data['correct_output'] = data['dec_out']
    data['predicted_output'] = output

    data = data.drop(['dec_inp', 'dec_out'], axis=1)
    
    return data

def compute_blue_score(results):
    data_output_bleu_score = []
    for encoder_input_data, correct_output , predicted_output in results.values:
        correct_output = correct_output.split()        
        predicted_output = predicted_output.rstrip().split()
        data_output_bleu_score.append(sentence_bleu([correct_output[:-1]], predicted_output))    
    blue_score=sum(data_output_bleu_score)/len(data_output_bleu_score)
    return blue_score

In [58]:
sample_data_train=train_data.sample(1000)
results=InferResults(sample_data_train)
blue_score=compute_blue_score(results)
print('BLEU score for Training data is: {}'.format(blue_score))
results.head(10)

BLUE score for Training data is: 0.7880080846981897


,enc_inp,correct_output,predicted_output
85373,as your teacher mr spock is fond of saying i l...,as your teacher mr spock is fond of saying i l...,as your teacher mr spock is fond of saying i l...
123494,me fall let us get some drugs,me fall let us get some drugs eos,me fall let us get some drugs
107511,you are trying to save sparazza,you are trying to save sparazza eos,you are trying to save sparazza
121157,no i will get you some tea wait till you are i...,no i will get you some tea wait till you are i...,no i will get you some tea wait till you are i...
47758,it is something really important jeff,it is something really important jeff eos,it is something really important jeff
47210,what is,what is eos,what is the
141372,that is very sweet of you vaughan we care abou...,that is very sweet of you vaughan we care abou...,that is very sweet of you vaughan we care abou...
22312,we will be expecting you,we will be expecting you eos,we will be expecting you
45323,how would you find it,how would you find it eos,how would you find it
70215,so how is it in there,so how is it in there eos,so how is it in there


In [59]:
sample_data_test=test_data.sample(1000)
results=InferResults(sample_data_test)
blue_score=compute_blue_score(results)
print('BLEU score for Test data is: {}'.format(blue_score))
results.head(10)

BLUE score for Test data is: 0.6357362905759035


,enc_inp,correct_output,predicted_output
19937,for that matter i think you will too,for that matter i think you will too eos,for that matter i think you will too
12333,but i thought he came over to america before war,but i thought he came over to america before t...,but i thought he came over to america before t...
30510,i were only asking if you are related to laval...,i was only asking if you are related to the la...,i was only asking if you are related to to the
17161,i afraid we have another problem mr president,i am afraid we have another problem mr preside...,i am afraid we have another problem mr president
15667,steady steady all right mr scott,steady steady all right mr scott eos,steady steady all right mr scott
10532,can we make breakaway speed,can we make breakaway speed eos,can we make breakaway speed
9605,let us,let us eos,let us
30393,i guess you do not get bothered by neighbors much,i guess you do not get bothered by neighbors m...,i guess you do not get bothered by the neighbo...
9317,yes i did,yes i did eos,yes i did
17582,nobody talking about ya moms,nobody talking about ya moms eos,nobody is talking about ya a he is


In [60]:
sample_data_validation=validation_data.sample(1000)
results=InferResults(sample_data_validation)
blue_score=compute_blue_score(results)
print('BLEU score for Validation data is: {}'.format(blue_score))
results.head(10)

BLUE score for Validation data is: 0.6684386187714704


,enc_inp,correct_output,predicted_output
38683,do you like quotes howard,do you like quotes howard eos,do you like the pets howard
26611,you know these doctors could have settled out ...,you know these doctors could have settled out ...,you know these doctors could have settled out ...
9462,joel you are a liar,joel you are a liar eos,joel you are a liar
17040,indeed they were deadly little things,indeed they are deadly little things eos,indeed they are a deadly little things
1569,this is extremely dangerous direction mr presi...,this is an extremely dangerous direction mr pr...,this is extremely dangerous direction mr presi...
12204,i do not expect you to be pleasant,i do not expect you to be pleasant eos,i do not expect you to be pleasant
12901,do not you know,do not you know eos,do not you know
25754,is it me is there something i have done,is it me is there something i have done eos,is it me is there something i have done
14209,i am sorry it is these goddamned shoes i do no...,i am sorry it is these goddamned shoes i do no...,i am sorry it is these goddamned shoes i do no...
18650,juliet who,juliet who eos,juliet who


## Observations
- The Bahadanau Monotonic attention model is proven to be good compared to basic encoder and decoder model
-The model gave good BLUE score of 0.788 on Train data, 0.6357 on Test data and 0.6684 on validation data
- To improve the model, we can add introduce more gramatical error in the data, train the model with different datasets, consider using more complex attention architecture and train on large data corpus
